In [11]:
# add your name here. Eg: yourname = 'vult5'
yourname = 'vult5'

# Example 1 - Lightweight component

Lightweight component is just like `dsl.ContainerOp`. <br>
The difference is it could transform a python function into a entrypoint for an defined image<br>

In [2]:
import kfp
from kfp import components, dsl
import easykubeflow as ekf

In [7]:
client = kfp.Client()
ekf.Utils(client)

In [3]:
@components.func_to_container_op
def print_op(message):
    from datetime import datetime
    print(f'{datetime.now()} -- message: {message}')

The approach above is not recommended in the restrict internet access environment<br>
cause the Kubeflow host would pull the default image that not appear in our Registry<br>
To replace default image, we have to do it like this

In [4]:
def print_op(message):
    from datetime import datetime
    print(f'{datetime.now()} -- message: {message}')
    
print_op = components.func_to_container_op(print_op, base_image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0')

To be more straightforward, the `easykubeflow` has the same function but allows<br>
you to define the base image in the decorator format

In [8]:
@ekf.func_to_container_op(base_image='asia.gcr.io/vinid-data-science-np/vu-example/base:1.0.0')
def print_op(message):
    from datetime import datetime
    print(f'{datetime.now()} -- message: {message}')

Now define sample pipeline 

In [10]:
@dsl.pipeline(
    name="Sample pipeline",
    description="Just a simple test for lightweight components"
)
def pipeline(yourname:str):
    print_op(yourname)

In [12]:
experiment=f'{yourname}-Examples'
run='Lightweight Component'

In [13]:
run_result = client.create_run_from_pipeline_func(pipeline_func=pipeline,
                                                  arguments={'yourname': yourname},
                                                  run_name=run,
                                                  experiment_name=experiment,
                                                  namespace='vult5')

In [14]:
client.runs.delete_run(run_result.run_id)

{}

# Example 2 - Recursion